In [2]:
import pandas as pd 
data_train = pd.read_excel('/content/drive/MyDrive/HASOC_2021/HASOC_Dravidian/Task2_Tamil_malayalam_code_mixed/Tamil_codemixed/Tamil-Codemixed_offensive_data_Training-Tweet-HL.xlsx', names=['id','text','label'])
data_val = pd.read_csv('/content/drive/MyDrive/HASOC_2021/HASOC_Dravidian/Task2_Tamil_malayalam_code_mixed/Tamil_codemixed/Tamil_hasoc_tanglish_test_withlabels.tsv', sep = '\t', names=['id','text','label'])
data_test = pd.read_csv('/content/drive/MyDrive/HASOC_2021/HASOC_Dravidian/Task2_Tamil_malayalam_code_mixed/Tamil_codemixed/hasoc_tamil_task2_withoutlabels.tsv', sep = '\t', names=['id','text'])

In [3]:
data_val = data_val.dropna()

In [4]:
data_train.shape, data_val.shape, data_test.shape

((3999, 3), (940, 3), (1001, 2))

In [5]:
data_train['label'].value_counts()

NOT    2018
OFF    1979
not       1
OFf       1
Name: label, dtype: int64

In [6]:
data_train['label'] = data_train['label'].replace('OFf', 'OFF')
data_train['label'] = data_train['label'].replace('not', 'NOT')

In [7]:
data_train['label'].value_counts()

NOT    2019
OFF    1980
Name: label, dtype: int64

In [8]:
data_val['label'] = data_val['label'].dropna()

In [9]:
data_val['label'].value_counts()

OFF    475
NOT    465
Name: label, dtype: int64

In [10]:
from sklearn import preprocessing
import numpy as np

le = preprocessing.LabelEncoder()

le.fit(data_train['label'])
print(le.classes_)

y_train_index = le.transform(data_train['label'])
y_val_index = le.transform(data_val['label'])


['NOT' 'OFF']


In [11]:
from keras.utils.np_utils import to_categorical
y_train_cal  =  to_categorical(y_train_index)
y_val_cal  =  to_categorical(y_val_index)


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvec = TfidfVectorizer(ngram_range=(1,6), analyzer = 'char_wb', max_features = 30000)
#tfidfvec = TfidfVectorizer(ngram_range=(1,3), max_features = 30000)
tfidf_train = tfidfvec.fit_transform(data_train['text'])
tfidf_val = tfidfvec.transform(data_val['text'])

In [13]:
x_train = tfidf_train
x_test = tfidf_val
y_train = y_train_cal
y_test =y_val_cal

In [14]:
from keras.models import Sequential 
from keras.layers import Dense, Dropout
model = Sequential()
model.add(Dense(4096, input_dim = tfidf_train.shape[1], activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4096)              122884096 
_________________________________________________________________
dropout (Dropout)            (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               2097664   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 125,014,722
Trainable params: 125,014,722
Non-trainable params: 0
__________________________________________

In [15]:
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [16]:
model.fit(x_train.toarray(), y_train, validation_data=(x_test.toarray(), y_test), epochs=10, batch_size=20, verbose=2 )

Epoch 1/10
200/200 - 19s - loss: 0.3741 - accuracy: 0.8285 - val_loss: 0.2715 - val_accuracy: 0.8798
Epoch 2/10
200/200 - 5s - loss: 0.1187 - accuracy: 0.9542 - val_loss: 0.4292 - val_accuracy: 0.8511
Epoch 3/10
200/200 - 5s - loss: 0.0516 - accuracy: 0.9827 - val_loss: 0.4074 - val_accuracy: 0.8660
Epoch 4/10
200/200 - 5s - loss: 0.0305 - accuracy: 0.9885 - val_loss: 0.6661 - val_accuracy: 0.8511
Epoch 5/10
200/200 - 5s - loss: 0.0184 - accuracy: 0.9897 - val_loss: 0.6024 - val_accuracy: 0.8649
Epoch 6/10
200/200 - 5s - loss: 0.0163 - accuracy: 0.9887 - val_loss: 0.6610 - val_accuracy: 0.8670
Epoch 7/10
200/200 - 5s - loss: 0.0172 - accuracy: 0.9902 - val_loss: 0.6467 - val_accuracy: 0.8670
Epoch 8/10
200/200 - 5s - loss: 0.0196 - accuracy: 0.9905 - val_loss: 0.8186 - val_accuracy: 0.8596
Epoch 9/10
200/200 - 5s - loss: 0.0208 - accuracy: 0.9900 - val_loss: 0.6346 - val_accuracy: 0.8670
Epoch 10/10
200/200 - 5s - loss: 0.0359 - accuracy: 0.9842 - val_loss: 0.6581 - val_accuracy: 0.875

In [17]:
tfidf_test = tfidfvec.transform(data_test['text'])

In [18]:
predictions_test_submission = model.predict(tfidf_test.toarray())

In [19]:
predictions_test_submission

array([[9.9999332e-01, 6.6406465e-06],
       [5.4449546e-01, 4.5550457e-01],
       [9.9997509e-01, 2.4926503e-05],
       ...,
       [6.6061182e-07, 9.9999928e-01],
       [7.8005050e-06, 9.9999225e-01],
       [1.0000000e+00, 8.4599130e-13]], dtype=float32)

In [20]:
import numpy as np
predictions_test_submission = np.asarray(predictions_test_submission)

In [21]:
import numpy as np
predictions_test_submission1 = np.zeros_like(predictions_test_submission)
predictions_test_submission1[np.arange(len(predictions_test_submission)), predictions_test_submission.argmax(1)] = 1

In [22]:
predictions_test_submission1

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [23]:
predictions_test_submission1_final = np.argmax(predictions_test_submission1, axis=1)

In [24]:
predictions_test_submission1_final1 = le.inverse_transform(predictions_test_submission1_final)

In [25]:
data_test['label'] = predictions_test_submission1_final1

In [26]:
data_test

,id,text,label
0,Tam_1,Indha movie ku award tharlana avanga mansanay ...,NOT
1,Tam_2,kritheeck Kookaburra en unaku enachu? Cbsc ah??,NOT
2,Tam_3,Actually Oru particular bus incident thalaiya ...,NOT
3,Tam_4,Small suggestions: mic ah shirt la pottukunga bro,NOT
4,Tam_5,Karnan padathulaa oru pombaa varumlaa athuu en...,OFF
...,...,...,...
996,Tam_997,Chai spoiler vaya mooduya full kathayum sillit...,OFF
997,Tam_998,Unakku thinga soru irukko illayo aduthavangala...,OFF
998,Tam_999,Dai seripu oli advingi mala vandavan thana mau...,OFF
999,Tam_1000,Gomala bule shirt ta nee padam eduthu paaruda ...,OFF


In [27]:
data_test.to_csv("/content/drive/MyDrive/HASOC_2021/HASOC_Dravidian/Task2_Tamil_malayalam_code_mixed/Tamil_codemixed/ANU_Task2_Tamil.tsv", sep="\t", index=False)

In [28]:
check_submission = pd.read_csv("/content/drive/MyDrive/HASOC_2021/HASOC_Dravidian/Task2_Tamil_malayalam_code_mixed/Tamil_codemixed/ANU_Task2_Tamil.tsv", sep="\t")

In [29]:
check_submission

,id,text,label
0,Tam_1,Indha movie ku award tharlana avanga mansanay ...,NOT
1,Tam_2,kritheeck Kookaburra en unaku enachu? Cbsc ah??,NOT
2,Tam_3,Actually Oru particular bus incident thalaiya ...,NOT
3,Tam_4,Small suggestions: mic ah shirt la pottukunga bro,NOT
4,Tam_5,Karnan padathulaa oru pombaa varumlaa athuu en...,OFF
...,...,...,...
996,Tam_997,Chai spoiler vaya mooduya full kathayum sillit...,OFF
997,Tam_998,Unakku thinga soru irukko illayo aduthavangala...,OFF
998,Tam_999,Dai seripu oli advingi mala vandavan thana mau...,OFF
999,Tam_1000,Gomala bule shirt ta nee padam eduthu paaruda ...,OFF
